###Advanced RAG Q&A Project With Multiple Data Sources With Langchain

In [1]:
%pip install arxiv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper   


In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=50)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/theabhi_13/Desktop/Project/ChatBot/.venv/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=50))

In [5]:
wiki.name

'wikipedia'

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
docs = loader.load()

In [8]:
RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(docs,HuggingFaceEmbeddings())
retriever=vectordb.as_retriever()
retriever


/var/folders/j7/29qrltcj4r7gwqtch93tsgf80000gn/T/ipykernel_19765/964343967.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  vectordb=FAISS.from_documents(docs,HuggingFaceEmbeddings())
/var/folders/j7/29qrltcj4r7gwqtch93tsgf80000gn/T/ipykernel_19765/964343967.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectordb=FAISS.from_documents(docs,HuggingFaceEmbeddings())


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10d1d01d0>, search_kwargs={})

In [11]:
from langchain_core.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever, "langsmith_search",
"search for imformation about langsmith. for any questions about langsmith, you need to answer question about langsmith.")

In [12]:
retriever_tool.name

'langsmith_search'

In [13]:
#Arxiv Tool-the website where the research papers are stored
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [14]:
#combining multiple tools
tools=[wiki,arxiv,retriever_tool]

In [15]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/theabhi_13/Desktop/Project/ChatBot/.venv/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=50)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 StructuredTool(name='langsmith_search', description='search for imformation about langsmith. for any questions about langsmith, you need to answer question about langsmith.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x10cc3a7a0>, coroutine=<function create_retriever_tool.<locals>.afunc at 0

In [17]:
from dotenv import load_dotenv
load_dotenv() 
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="mistralai/mistral-7b-instruct",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENROUTER_API_KEY"],
    temperature=0.7,
    default_headers={
        "HTTP-Referer": "http://localhost",   # REQUIRED by OpenRouter
        "X-Title": "LangChain-RAG-App"         # REQUIRED by OpenRouter
    }
)

In [ ]:
import  langchainhub
#get the prompt to use - you can modify this to use your own
prompt=langchainhub.pull("hwchase17/openai-functions-agent")
prompt.messages


In [ ]:
#Agents
from langchain.agents import create_structured_chat_agent, AgentExecutor
from langchain import hub
agent= create_structured_chat_agent(
    llm=llm,
    tools=tool,
    prompt=prompt
)


ImportError: cannot import name 'create_structured_chat_agent' from 'langchain.agents' (/Users/theabhi_13/Desktop/Project/ChatBot/.venv/lib/python3.11/site-packages/langchain/agents/__init__.py)

In [4]:
#agent Executor 
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True
)


ImportError: cannot import name 'AgentExecutor' from 'langchain.agents' (/Users/theabhi_13/Desktop/Project/ChatBot/.venv/lib/python3.11/site-packages/langchain/agents/__init__.py)

In [5]:
agent_executor.invoke("input" : "Tell me about langsmith")

SyntaxError: invalid syntax (626651270.py, line 1)